In [1]:
!pip install pandas openpyxl scikit-learn imbalanced-learn shap tensorflow
print("✅ Libraries installed successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 73.2 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
✅ Libraries installed successfully.


In [2]:
import pandas as pd

# The path to your CSV file
file_name = "/kaggle/input/fakejobposting3/fake_job_postings.csv" 

# Use pd.read_csv() instead of pd.read_excel()
df = pd.read_csv(file_name)

print(f"\n✅ Successfully loaded: {file_name}")
print(df.head())


✅ Successfully loaded: /kaggle/input/fakejobposting3/fake_job_postings.csv
   job_id                                      title            location  \
0       1                           Marketing Intern    US, NY, New York   
1       2  Customer Service - Cloud Video Production      NZ, , Auckland   
2       3    Commissioning Machinery Assistant (CMA)       US, IA, Wever   
3       4          Account Executive - Washington DC  US, DC, Washington   
4       5                        Bill Review Manager  US, FL, Fort Worth   

  department salary_range                                    company_profile  \
0  Marketing          NaN  We're Food52, and we've created a groundbreaki...   
1    Success          NaN  90 Seconds, the worlds Cloud Video Production ...   
2        NaN          NaN  Valor Services provides Workforce Solutions th...   
3      Sales          NaN  Our passion for improving quality of life thro...   
4        NaN          NaN  SpotSource Solutions LLC is a Global Hum

In [3]:
print("--- Phase 1: Feature Engineering ---")

# a. Create "missingness" features
df['is_company_profile_missing'] = df['company_profile'].isnull().astype(int)
df['is_requirements_missing'] = df['requirements'].isnull().astype(int)
df['is_benefits_missing'] = df['benefits'].isnull().astype(int)
df['is_salary_range_missing'] = df['salary_range'].isnull().astype(int)

# b. Combine all text fields into one for analysis
text_columns = ['title', 'location', 'department', 'company_profile', 
                'description', 'requirements', 'benefits', 'function']

# Fill NaNs with empty strings to avoid errors
for col in text_columns:
    df[col] = df[col].fillna('')

df['text_combined'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

print("✅ New features created:")
print(df[['is_company_profile_missing', 'is_salary_range_missing', 'text_combined']].head())

--- Phase 1: Feature Engineering ---
✅ New features created:
   is_company_profile_missing  is_salary_range_missing  \
0                           0                        1   
1                           0                        1   
2                           0                        1   
3                           0                        1   
4                           0                        1   

                                       text_combined  
0  Marketing Intern US, NY, New York Marketing We...  
1  Customer Service - Cloud Video Production NZ, ...  
2  Commissioning Machinery Assistant (CMA) US, IA...  
3  Account Executive - Washington DC US, DC, Wash...  
4  Bill Review Manager US, FL, Fort Worth  SpotSo...  


In [4]:
def clean_text(text):
    """
    Cleans a raw text string:
    1. Lowercases
    2. Removes HTML tags
    3. Removes punctuation and numbers
    4. Tokenizes
    5. Removes stopwords
    """
    text = text.lower()  # Lowercase
    text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Remove punctuation/numbers
    tokens = word_tokenize(text)  # Tokenize
    # Remove stop words
    cleaned_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(cleaned_tokens)

print("✅ `clean_text` function defined.")

✅ `clean_text` function defined.


In [5]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
# --- Standard Libraries ---
import pandas as pd
import numpy as np
import re
import warnings

# --- NLTK for Text Processing ---
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- Scikit-learn (sklearn) ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# --- TensorFlow & Keras (for
# CNN+LSTM) ---
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, concatenate, Dropout

# --- Setup ---
warnings.filterwarnings('ignore') # Suppress warnings for cleaner output
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
stop_words = set(stopwords.words('english'))

print("✅ All libraries imported and NLTK data downloaded. Ready for CNN+LSTM.")

2025-10-31 12:50:46.880529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761915047.215769      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761915047.312200      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ All libraries imported and NLTK data downloaded. Ready for CNN+LSTM.


In [7]:

print("Cleaning all text data... (This may take a moment)")
df['text_cleaned'] = df['text_combined'].apply(clean_text)
print("✅ Text cleaning complete.")
print(df[['text_combined', 'text_cleaned']].head())

Cleaning all text data... (This may take a moment)
✅ Text cleaning complete.
                                       text_combined  \
0  Marketing Intern US, NY, New York Marketing We...   
1  Customer Service - Cloud Video Production NZ, ...   
2  Commissioning Machinery Assistant (CMA) US, IA...   
3  Account Executive - Washington DC US, DC, Wash...   
4  Bill Review Manager US, FL, Fort Worth  SpotSo...   

                                        text_cleaned  
0  marketing intern new york marketing food creat...  
1  customer service cloud video production auckla...  
2  commissioning machinery assistant cma wever va...  
3  account executive washington washington sales ...  
4  bill review manager fort worth spotsource solu...  


In [8]:
print("--- Phase 2: Defining Features & Target ---")

target = 'fraudulent'

# Define which columns go into which preprocessing step
binary_features = [
    'telecommuting', 'has_company_logo', 'has_questions', 
    'is_company_profile_missing', 'is_requirements_missing', 
    'is_benefits_missing', 'is_salary_range_missing'
]
categorical_features = ['employment_type', 'required_experience', 'required_education', 'industry']
text_feature = 'text_cleaned' # Our preprocessed text

# Handle missing values in categorical features *before* splitting
df[categorical_features] = df[categorical_features].fillna('Missing')

# Create X and y
X = df[binary_features + categorical_features + [text_feature]]
y = df[target]

print(f"✅ X (features) and y (target) are defined.")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

--- Phase 2: Defining Features & Target ---
✅ X (features) and y (target) are defined.
Shape of X: (17880, 12)
Shape of y: (17880,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,    # 30% of data for testing
    random_state=42,  # Ensures reproducible results
    stratify=y        # Keeps the same % of fake jobs in train and test
)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training target distribution:\n{y_train.value_counts(normalize=True)}")
print(f"Testing target distribution:\n{y_test.value_counts(normalize=True)}")

Training set shape: (12516, 12)
Testing set shape: (5364, 12)
Training target distribution:
fraudulent
0    0.951582
1    0.048418
Name: proportion, dtype: float64
Testing target distribution:
fraudulent
0    0.951529
1    0.048471
Name: proportion, dtype: float64


In [10]:
# --- Keras Text Preprocessing Settings ---
# We define these constants here for our new text pipeline
MAX_VOCAB_SIZE = 20000 # Max words to keep in vocabulary
MAX_SEQUENCE_LENGTH = 250 # Max length of a job description

# --- Keras Tabular Preprocessing ---
# We create a preprocessor *only* for the non-text features.
# The text feature will be handled separately by the Keras Tokenizer.

tabular_preprocessor = ColumnTransformer(
    transformers=[
        # (name, transformer, columns_to_apply_to)
        # Use sparse_output=False so Keras gets a dense array
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('binary', 'passthrough', binary_features) # 'passthrough' leaves these columns as-is
    ],
    remainder='drop' # Drop any columns not specified
)

print("✅ Tabular preprocessor created.")
print(f"✅ Text constants defined (MAX_VOCAB_SIZE={MAX_VOCAB_SIZE}, MAX_SEQUENCE_LENGTH={MAX_SEQUENCE_LENGTH}).")

✅ Tabular preprocessor created.
✅ Text constants defined (MAX_VOCAB_SIZE=20000, MAX_SEQUENCE_LENGTH=250).


In [11]:
# === 1. Prepare Text Data (Tokenizer) ===
print("Preparing text data...")
# Initialize Keras Tokenizer
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
# Fit on training text
tokenizer.fit_on_texts(X_train[text_feature])

# Convert text to padded sequences
X_train_seq = tokenizer.texts_to_sequences(X_train[text_feature])
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

X_test_seq = tokenizer.texts_to_sequences(X_test[text_feature])
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# The true vocabulary size (including OOV and padding)
vocab_size = len(tokenizer.word_index) + 1
print(f"Text data prepared. Vocab size: {vocab_size}, Sequence length: {MAX_SEQUENCE_LENGTH}")

# === 2. Prepare Tabular Data (ColumnTransformer) ===
print("Preparing tabular data...")
# Get column names for the tabular preprocessor
tabular_features = categorical_features + binary_features

# Fit and transform the training data
X_train_tab = tabular_preprocessor.fit_transform(X_train[tabular_features])
# Transform the test data
X_test_tab = tabular_preprocessor.transform(X_test[tabular_features])

# Get shape of the processed tabular data
tabular_shape = X_train_tab.shape[1]
print(f"Tabular data prepared. Input shape: ({tabular_shape},)")

# === 3. Define the CNN + LSTM Model ===
print("Building the model...")
EMBEDDING_DIM = 100

# --- Text Input Branch ---
text_input = Input(shape=(MAX_SEQUENCE_LENGTH,), name='text_input')
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH)(text_input)

# CNN part
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
pool_layer = GlobalMaxPooling1D()(conv_layer)

# LSTM part (re-uses the embedding layer as input)
lstm_layer = LSTM(units=128, dropout=0.2)(embedding_layer)

# Combine text branches
text_branch = concatenate([pool_layer, lstm_layer])
text_branch = Dropout(0.5)(text_branch)

# --- Tabular Input Branch ---
tabular_input = Input(shape=(tabular_shape,), name='tabular_input')
tabular_branch = Dense(64, activation='relu')(tabular_input)
tabular_branch = Dropout(0.3)(tabular_branch)

# --- Merge All Branches ---
merged = concatenate([text_branch, tabular_branch])
merged_dense = Dense(128, activation='relu')(merged)
merged_dense = Dropout(0.5)(merged_dense)
output = Dense(1, activation='sigmoid', name='output')(merged_dense) # Sigmoid for binary classification

# --- Create and Compile Model ---
model = Model(inputs=[text_input, tabular_input], outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

# === 4. Handle Class Imbalance (Class Weights) ===
# We use class weights instead of SMOTE, which is much simpler for Keras
print("Calculating class weights...")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weight_dict}")

# === 5. Train the Model ===
print("Training the model...")
history = model.fit(
    [X_train_pad, X_train_tab],  # Pass inputs as a list
    y_train,
    epochs=3,                    # Keep low for a quick test
    batch_size=64,
    validation_split=0.1,
    class_weight=class_weight_dict # Pass the weights here
)
print("✅ Model training complete.")

Preparing text data...
Text data prepared. Vocab size: 78970, Sequence length: 250
Preparing tabular data...
Tabular data prepared. Input shape: (165,)
Building the model...


2025-10-31 12:51:35.463726: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 250)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 250, 100)  │  7,897,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 246, 128)  │     64,128 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │    117,248 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabular_input       │ (None, 165)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     10,624 │ tabular_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 320)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     41,088 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        129 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,130,217 (31.01 MB)

 Trainable params: 8,130,217 (31.01 MB)

 Non-trainable params: 0 (0.00 B)

Calculating class weights...
Class weights: {0: 0.52544080604534, 1: 10.326732673267326}
Training the model...
Epoch 1/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 75s 392ms/step - accuracy: 0.8012 - loss: 0.5916 - precision: 0.1292 - recall: 0.5558 - val_accuracy: 0.8986 - val_loss: 0.2487 - val_precision: 0.2800 - val_recall: 0.9800
Epoch 2/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 67s 379ms/step - accuracy: 0.9246 - loss: 0.1681 - precision: 0.3821 - recall: 0.9461 - val_accuracy: 0.9457 - val_loss: 0.1377 - val_precision: 0.4196 - val_recall: 0.9400
Epoch 3/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 67s 379ms/step - accuracy: 0.9748 - loss: 0.0625 - precision: 0.6594 - recall: 0.9896 - val_accuracy: 0.9704 - val_loss: 0.0696 - val_precision: 0.5867 - val_recall: 0.8800
✅ Model training complete.


In [12]:
print("Evaluating the model on the test set...")

# Predict probabilities (Keras model outputs probabilities)
y_pred_proba = model.predict([X_test_pad, X_test_tab])

# Convert probabilities to binary classes (0 or 1) using a 0.5 threshold
y_pred = (y_pred_proba > 0.5).astype(int)

# --- Classification Report ---
# Pay close attention to the precision and recall for "Fake (1)"
# Precision (Fake): Of all jobs we flagged as fake, what % was *actually* fake?
# Recall (Fake): Of all *actual* fake jobs, what % did we *catch*?
print(classification_report(y_test, y_pred, target_names=['Real (0)', 'Fake (1)']))

Evaluating the model on the test set...
168/168 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step
              precision    recall  f1-score   support

    Real (0)       0.99      0.98      0.98      5104
    Fake (1)       0.65      0.88      0.75       260

    accuracy                           0.97      5364
   macro avg       0.82      0.93      0.87      5364
weighted avg       0.98      0.97      0.97      5364

